<a href="https://colab.research.google.com/github/RUROBO/YOLOv4_Colab/blob/main/yolov4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#YOLOv4-Cloud-Colab-教程

###点击装载Google云端硬盘，或者直接在左边的命令行输入指令进行装载，指令如下：



*   将.ipynb文件装载谷歌云盘，方便Googel云盘和Colab分配的云GPU进行文件相互传输块




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#1.git clone darknet【克隆github上面的darknet】

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

#### 修改makefile 将OpenCV和GPU设置为可用

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
#验证CUDA版本
!/usr/local/cuda/bin/nvcc --version

#2.编译项目

In [ ]:
!make

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#将谷歌云盘路径简写为mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
!ls /mydrive/yolov4

In [ ]:
#将谷歌云盘中的yolov4上传到darknet目录下
!cp /mydrive/yolov4/yolov4-tiny.weights ./

In [ ]:
#定义imshow 调用opencv显示图片
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4-tiny.cfg yolov4-tiny.weights  data/dog.jpg

In [ ]:
imShow('predictions.jpg')

In [ ]:
#确保当前路径在darknet下
!ls
#将数据集图片及标注的txt文件的压缩包上传到服务器
!cp /mydrive/yolov4/obj.zip ../
#解压到data文件下
!unzip ../obj.zip -d data/

In [ ]:
#从谷歌云盘上传根据自己数据集修改的.cfg文件
!cp /mydrive/yolov4/yolov4_custom.cfg ./cfg

In [ ]:
!cp /mydrive/yolov4/yolov4.conv.137 ./

In [ ]:
#从谷歌云盘上传自己数据集的obj.data obj.names
!cp /mydrive/yolov4/obj.names ./data
!cp /mydrive/yolov4/obj.data  ./data

In [ ]:
#上传generate_train.py 以在服务器的data下生成train.txt
!cp /mydrive/yolov4/generate_train.py ./
!cp /mydrive/yolov4/generate_test.py ./

In [ ]:
!python generate_train.py
!python generate_test.py

##防止运行终止
To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```



#开始训练

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4_custom.cfg yolov4.conv.137 -dont_show -map

##实际在训练中加-map可能导致出错，可以选择训练不加-map，训练结束后再采用如下指令计算权重文件map：

In [ ]:
#不加-iou_thresh 默认计算map@50，即iou=0.5
!./darknet detector map data/obj.data cfg/yolov4_custom.cfg /mydrive/yolov4/backup/yolov4_custom_final.weights
#map@75
!./darknet detector map data/obj.data cfg/yolov4_custom.cfg /mydrive/yolov4/backup/yolov4_custom_final.weights -iou_thresh 0.75

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4_custom.cfg darknet53.conv.74 -dont_show

#终端的命令

In [ ]:
!cp /mydrive/yolov4/backup/yolov4_custom_last.weights ./backup

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4_custom.cfg backup/yolov4_custom_last.weights  -dont_show

In [ ]:
!cp /mydrive/yolov4/test.jpg  ./data/obj/

In [ ]:
#图片测试
!./darknet detector test data/obj.data cfg/yolov4_custom.cfg /mydrive/yolov4/backup/yolov4_custom_final.weights ./data/obj/0001010_jpg.rf.43520f29eab30a3282b878792b8f8ae0.jpg -thresh 0.5
imShow('predictions.jpg')

#视频检测

In [ ]:
#视频测试
!./darknet detector demo data/obj.data cfg/yolov4_custom.cfg /mydrive/yolov4/backup/yolov4_custom_final.weights -dont_show /mydrive/videos/test.mp4 -i 0 -out_filename /mydrive/videos/菠萝.avi

In [ ]:
import numpy as np
import time
import cv2
import os
 
labelsPath = "data/obj.data/obj.names"
LABELS = pineapple
with open(labelsPath, 'rt') as f:
    LABELS = f.read().rstrip('n').split("n")
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
                           dtype="uint8")
weightsPath = "/mydrive/yolov4/backup/yolov4_custom_final.weights"
configPath = "cfg/yolov4_custom.cfg" 
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]